## Data enhancement

In [57]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

### Pre-processing images in data
** note that only training data will undergo enhancement

#### Image enhancement
The process of improving the appearance of images to highlight specific features, reduce noise or improve the quality of the image, this helps it to be more suitable for analysis.

Alpha focuses on the contrast of image
- alpha greater than 1: images brighter, enhanced contrast
- alpha less than 1: images darker, reduced contrast

Beta focuses on brightness of image
- beta positive: makes images brighter
- beta negative: makes images darker

In [39]:
train_data = pd.read_csv("datasets/csv/train_data.csv")
test_data = pd.read_csv("datasets/csv/test_data.csv")

## PREPROCESSING IMAGES WITH ENHANCEMENT
def preprocess_image(img_path, enhance):
    # reading images
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if enhance == True:
        # enhancing image
        img = cv2.convertScaleAbs(img, alpha = 1.5, beta = -20)
    # target_size of 224, 224 commonly used for image classification
    img = cv2.resize(img, (224, 224))
    # normalising pixel values
    img_array = img.astype(np.float32) / 255
    return img_array

In [40]:
# using preprocessed images as train data
train_images = np.array([preprocess_image(image_path, True) for image_path in train_data["image_path"]])
# using "pathology" column as train labels
train_labels = np.array(train_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
train_labels[train_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

In [41]:
# using preprocessed images as test data
test_images = np.array([preprocess_image(image_path, False) for image_path in test_data["image_path"]])
# using "pathology" column as test labels
test_labels = np.array(test_data["pathology"])

# change "BENIGN_WITHOUT_CALLBACK" to "BENIGN"
test_labels[test_labels == "BENIGN_WITHOUT_CALLBACK"] = "BENIGN"

#### Image augmentation
```ImageDataGenerator``` provides real-time data augmentation. Generates augmented images on-the-fly. For this project it is fixed so just need horizontal and vertical flips. To ensure that at least basic method of flipping works, a Numpy version will be implemented followed by the ```ImageDataGenerator``` version, this also tests which version of augmentation is better.

In [ ]:
## VERSION 1 OF IMAGE AUGMENTATION - using numpy
augmented_images = []
augmented_labels = []

# augmenting images and storing in lists
for i, img_path in enumerate(train_data["image_path"]):
    original_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    enhanced_image = preprocess_image(img_path, True)
    
    # adding original image and label
    augmented_images.append(enhanced_image)
    augmented_labels.append(train_labels[i])
    
    # making all combinations of flips
    for horizontal_flip in [True, False]:
        for vertical_flip in [True, False]:
            # applying flips
            augmented_image = enhanced_image
            if horizontal_flip:
                augmented_image = np.fliplr(augmented_image)
            if vertical_flip:
                augmented_image = np.flipud(augmented_image)
                
            # adding augmented image and label
            augmented_images.append(augmented_image)
            augmented_labels.append(train_labels[i])
        
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

In [ ]:
# encoding labels
label_encoder = LabelEncoder()
encoded_train_labels = label_encoder.fit_transform(train_labels)
encoded_aug_train_labels = label_encoder.fit_transform(augmented_labels)
encoded_test_labels = label_encoder.fit_transform(test_labels)
# one-hot encode labels
one_hot_train_labels = tf.keras.utils.to_categorical(encoded_train_labels)
one_hot_aug_train_labels = tf.keras.utils.to_categorical(encoded_aug_train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(encoded_test_labels)

In [ ]:
## VERSION 2 OF IMAGE AUGMENTATION - using TensorFlow
datagen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = "nearest"
)

# version 2 of augmenting images
augmented_generator = datagen.flow(train_images[:, :, :, np.newaxis], one_hot_train_labels, batch_size = 28)

## Creating of models

In [ ]:
base_model1 = Sequential()
# creating stack of Conv2D and MaxPooling2D
base_model1.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
base_model1.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
base_model1.add(Flatten())
base_model1.add(Dense(128, activation = "relu"))
# output layer with softmax
base_model1.add(Dense(2, activation = "softmax"))

base_model2 = Sequential()
# creating stack of Conv2D and MaxPooling2D
base_model2.add(Conv2D(32, (3, 3), activation = "relu", input_shape = (224, 224, 1)))
base_model2.add(MaxPooling2D((2, 2)))

# unrolling output to 1D
base_model2.add(Flatten())
base_model2.add(Dense(128, activation = "relu"))
# output layer with softmax
base_model2.add(Dense(2, activation = "softmax"))

It was noted that whilst the adam optimizer had the highest accuracy, the nadam optimizer has a higher validation accuracy, which would suggest a change in choice to the preliminary report.

In [ ]:
# compile model, improving accuracy
base_model1.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = base_model1.fit(augmented_images, one_hot_aug_train_labels, epochs = 10, validation_data = (test_images, one_hot_test_labels))

In [ ]:
# compile model, improving accuracy
base_model2.compile(optimizer = "Nadam", loss = "categorical_crossentropy", metrics = ["accuracy"])
# train model, validating on test set
history = base_model2.fit(augmented_generator, epochs = 10, validation_data = (test_images[:, :, :, np.newaxis], one_hot_test_labels))